# Vehicle Emissions Breakdown, ICE vs EV per Power Generation Method

Two independent variables: power generation method (wind/solar/coal/gas/etc.) and vehicle type (ICE/EV). The dependent variable is grams of CO2 emitted per mile.

In [1]:
################################################################################
#  IMPORTS                                                                     #
################################################################################
using DataFrames
using CSV

include("demand_functions.jl")

MWh_vec_to_CO2_vec

In [2]:
################################################################################
#  CONSTANTS                                                                   #
################################################################################

# Emissions data for power plants
EMP = power_plant_emissions_profile()

# Output data file
OUTPUT_FILE = "emissions_profile.csv"

"emissions_profile.csv"

In [ ]:
################################################################################
#  FUNCTIONS                                                                   #
################################################################################

"""
    ICE_emissions_per_mile

Returns the emissions per mile for an ICE vehicle. Sourced from EPA aggregate
data. Units grams of CO2 per mile.
"""
function ICE_emissions_per_mile()
    return 404.0
end

"""
    EV_emissions_per_mile(power_profile::NamedTuple{PLANT_CATEGORIES, 
                                     NTuple{length(PLANT_CATEGORIES), Float64}})

Returns the emissions per mile for an EV based on the a given profile of power
production. Units grams of CO2 per mile.
"""
function EV_emissions_per_mile(power_profile::NamedTuple{PLANT_CATEGORIES, 
                                     NTuple{length(PLANT_CATEGORIES), Float64}})
    # Power requirement for an EV per mile
    # 189.2 Wh / Km * 1.60934 Km / mile / 1e6 Wh / MWh = 3.045e-4 MWh / mile
    power_per_mile = 3.045e-4

    # Convert power profile to power per mile
    power_profile_per_mile = plant_pcts_to_MWh_vec(power_profile,
                                                                 power_per_mile)

    # Compute the emissions for this power level based on the power profile
    emissions_per_mile = MWh_vec_to_CO2_vec(power_profile_per_mile, EMP)

    # Convert from tons of CO2 to grams of CO2 and return
    emissions_per_mile_grams = NamedTuple{keys(emissions_per_mile)}(
                             map(x -> x * 100000.0, values(emissions_per_mile)))
    return emissions_per_mile_grams
end

"""
    EV_emissions_per_renewable_pct(pct_renewable::Float64)

Returns the emissions per mile for an EV based on the percentage of renewable
energy in the power grid. Units grams of CO2 per mile.
"""
function EV_emissions_per_renewable_pct(pct_renewable::Float64)
    # Compute renewable and non-renewable fractions per fuel type for a given
    # renewable percentage
    RE_pct = pct_renewable / 6.0         # 6 types of renewable energy
    NRE_pct = (1 - pct_renewable) / 5.0  # 5 types of non-renewable energy
    return sum(values(EV_emissions_per_mile((;
               :BIOMASS    => RE_pct,
               :COAL       => NRE_pct,
               :GAS        => NRE_pct,
               :GEOTHERMAL => RE_pct,
               :HYDRO      => RE_pct,
               :NUCLEAR    => RE_pct,
               :OFSL       => NRE_pct,
               :OIL        => NRE_pct,
               :OTHF       => NRE_pct,
               :SOLAR      => RE_pct,
               :WIND       => RE_pct
    )))) 
end

EV_emissions_per_renewable_pct

In [20]:
################################################################################
#  MAIN                                                                        #
################################################################################

# Collect the total CO2 emissions for each singlet power profile (i.e, 100% of 
# power from one plant type) and add them to a new dataframe
plant_singlet_emissions = DataFrame(PlantType=String[],
                                    emissions_grams_per_mile=Float64[],
                                    percent_of_economy=Float64[])
total_samples = sum(EMP.Samples)
for plant_type in PLANT_CATEGORIES
    zero_profile = NamedTuple{PLANT_CATEGORIES}(ntuple(_ -> 0.0,
                                                      length(PLANT_CATEGORIES)))
    power_profile = (; zero_profile..., plant_type => 1.0)
    plant_row_idx = findall(EMP.PlantType .== string(plant_type))[1]
    pct = EMP.Samples[plant_row_idx] / total_samples
    push!(plant_singlet_emissions, (PlantType=string(plant_type),
      emissions_grams_per_mile=EV_emissions_per_mile(power_profile)[plant_type],
      percent_of_economy=pct))
end

# Add the ICE emissions (conventional gas) per mile to the overall data set 
# and output to CSV

ice_emissions = ICE_emissions_per_mile()
push!(plant_singlet_emissions, (
    PlantType="ICE",
    emissions_grams_per_mile=ice_emissions,
    percent_of_economy=0.0  # ICE vehicles are not part of the power economy
    )
)

println(plant_singlet_emissions)

CSV.write(OUTPUT_FILE, plant_singlet_emissions)


12×3 DataFrame
 Row │ PlantType   emissions_grams_per_mile  percent_of_economy 
     │ String      Float64                   Float64            
─────┼──────────────────────────────────────────────────────────
   1 │ BIOMASS                    76.6787            0.0497938
   2 │ COAL                      309.126             0.0176023
   3 │ GAS                       140.178             0.158024
   4 │ GEOTHERMAL                  8.9367            0.0053124
   5 │ HYDRO                       0.349993          0.115683
   6 │ NUCLEAR                    12.8681            0.00428164
   7 │ OFSL                      182.328             0.00142721
   8 │ OIL                       244.464             0.0696162
   9 │ OTHF                      343.596             0.0238662
  10 │ SOLAR                      47.9799            0.44862
  11 │ WIND                        0.149448          0.105772
  12 │ ICE                       404.0               0.0


"emissions_profile.csv"